# AoC 2022 Day 16 - Volcanos

## Part one

Given a network of valves and tunnels, and a time constraint, what is the max pressure that can be released?

- moving through one tunnel takes 1 minute
- opening takes 1 minute
- 30 minutes total

In [16]:
def load_input(fp):
    with open(fp) as f_in:
        for line in f_in.read().splitlines():
            yield line

In [17]:
# max for test = 1651
test = [line for line in load_input("test.txt")]

In [18]:
test[:3]

['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
 'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
 'Valve CC has flow rate=2; tunnels lead to valves DD, BB']

Build a graph from the inputs.

- each node has at least one children
- each node has a `release` value
- graph may be cyclic; one's children node may eventually lead back to the original node

In [19]:
class Valve:
    def __init__(self, name='', rate=0, neighbors=None, visited=False):
        """To use in defaultdict, must set defaults"""
        self.name = name
        self.rate = rate
        self.visited = visited
        if neighbors:
            self.neighbors = set(neighbors)
        else:
            self.neighbors = None
            
        
    def __repr__(self):
        return self.name
            

In [20]:
foo = Valve('AA', 3, ['DD', 'II', 'BB'])
print(foo.name, foo.rate, foo.neighbors)

AA 3 {'DD', 'BB', 'II'}


In [21]:
line = test[0]
spl = line.split(" ")
spl

['Valve',
 'AA',
 'has',
 'flow',
 'rate=0;',
 'tunnels',
 'lead',
 'to',
 'valves',
 'DD,',
 'II,',
 'BB']

In [22]:
import re

In [23]:
c = re.compile('Valve ([A-Z]{2}).*rate=(\d+).*valves (.*)')
child_c = re.compile('[A-Z]{2}')
res = c.findall(line)
child_c.findall(res[0][-1])

['DD', 'II', 'BB']

In [24]:
v = re.compile('[A-Z]{2}')
r = re.compile('rate=(\d+)')
v_m = v.findall(line)
r_m = r.findall(line)
print(v_m, r_m)

['AA', 'DD', 'II', 'BB'] ['0']


In [25]:
def parse_valve_tunnel(line: str) -> Valve:
    """Parse input to return 
    - valve name: str
    - rate: int
    - neighbors: list[str]
    
    in the form of a Valve object
    """
    v = re.compile('[A-Z]{2}')
    r = re.compile('rate=(\d+)')
    v_m = v.findall(line)
    r_m = r.findall(line)
    return Valve(v_m[0], int(r_m[0]), v_m[1:])
    

`defaultdict(list)` is better.

- `graph['AA'] = [neighbors]`
- `rate['AA'] = rate_AA`

To do so with a class...

```py
graph['AA'].neighbors = [neighbors]
graph['AA'].rate = rate_AA
```

key: valve_ID; value: `Valve` object

perhaps a `defaultdict(node)`.

In [28]:
from collections import defaultdict

tunnels = defaultdict(Valve)

In [29]:
fp = "test.txt"
tunnels = {valve.name: valve
            for line in load_input(fp)
            if (valve := parse_valve_tunnel(line))}

In [30]:
tunnels['AA']

AA

In [31]:
from functools import reduce


In [32]:
valve_time = {
    'BB': 1,
    'DD': 5
}
 
print(tunnels['BB'].rate, tunnels['DD'].rate)

13 20


In [33]:
tot = reduce(
    lambda subtot, valve: subtot + valve_time[valve] * tunnels[valve].rate, 
    valve_time, 0)
tot

113

In [34]:
from collections import deque

In [35]:
def dijkstra_valves(graph: dict, source: str) -> dict:
    """
    Find shortest paths between all pairs of vertices.
    Assumes all neighboring nodes have distance=1
    
    Parameters
    ---------
    graph: dict
        dict of all nodes, with Node object as the value
    source: str
        name of source node
    
    Returns
    -------
    shortest_paths: dict
        keyed by nodes, the dict contains the shortest distance to each neighboring node
    """
    # print(f'source: {source}')
    # mark all nodes really far, and mark all nodes unvisited
    dist = defaultdict(lambda: 1000) # {node1: dist1, node2: dist2, ...}
    unvisited = list(graph.keys())
    # prev = defaultdict(lambda: "")
        
    # print(f'unvisited:\n{unvisited}')
    # 2. assign distance=0 to origin
    dist[source] = 0
    
    while unvisited:
        # pop node with lowest dist
        current = sorted(unvisited, key=lambda x: dist[x])[0]
        unvisited.remove(current)
        
        # consider each neighbor
        # print(f'current: {current}\tneighbors: {graph[current].neighbors}')
        for neighbor in graph[current].neighbors:
            if neighbor in unvisited:
                
                # check if new dist is smaller than existing
                if dist[neighbor] > (new_dist := dist[current] + 1):
                    dist[neighbor] = new_dist
                    # prev[neighbor] = current
            
    return dist

In [37]:
dist = dijkstra_valves(tunnels, 'AA')

In [38]:
dists = {valve: dijkstra_valves(tunnels, valve) for valve in tunnels}

In [39]:
dists['JJ']

defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,
            {'JJ': 0,
             'AA': 2,
             'BB': 3,
             'CC': 4,
             'DD': 3,
             'EE': 4,
             'FF': 5,
             'GG': 6,
             'HH': 7,
             'II': 1})

In [40]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall, dijkstra

Requires building a matrix to represent the graphs

In [41]:
a_map = {i: valve_id for i, valve_id in enumerate(tunnels)}
a_map[1]

'BB'

In [42]:
import numpy as np

In [43]:
num_v = len(tunnels)
adj = np.empty(shape=(num_v, num_v), dtype=bool)

In [45]:
for i in range(num_v):
    for j in range(num_v):
        adj[i, j] = a_map[j] in tunnels[a_map[i]].neighbors
        
adj

array([[False,  True, False,  True, False, False, False, False,  True,
        False],
       [ True, False,  True, False, False, False, False, False, False,
        False],
       [False,  True, False,  True, False, False, False, False, False,
        False],
       [ True, False,  True, False,  True, False, False, False, False,
        False],
       [False, False, False,  True, False,  True, False, False, False,
        False],
       [False, False, False, False,  True, False,  True, False, False,
        False],
       [False, False, False, False, False,  True, False,  True, False,
        False],
       [False, False, False, False, False, False,  True, False, False,
        False],
       [ True, False, False, False, False, False, False, False, False,
         True],
       [False, False, False, False, False, False, False, False,  True,
        False]])

In [46]:
adj = csr_matrix(adj)

In [47]:
dist_mat = floyd_warshall(
    csgraph=adj, 
    directed=False, 
    return_predecessors=False,
    unweighted=False)
dist_mat

array([[0., 1., 2., 1., 2., 3., 4., 5., 1., 2.],
       [1., 0., 1., 2., 3., 4., 5., 6., 2., 3.],
       [2., 1., 0., 1., 2., 3., 4., 5., 3., 4.],
       [1., 2., 1., 0., 1., 2., 3., 4., 2., 3.],
       [2., 3., 2., 1., 0., 1., 2., 3., 3., 4.],
       [3., 4., 3., 2., 1., 0., 1., 2., 4., 5.],
       [4., 5., 4., 3., 2., 1., 0., 1., 5., 6.],
       [5., 6., 5., 4., 3., 2., 1., 0., 6., 7.],
       [1., 2., 3., 2., 3., 4., 5., 6., 0., 1.],
       [2., 3., 4., 3., 4., 5., 6., 7., 1., 0.]])

In [50]:
dists

{'AA': defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,
             {'AA': 0,
              'BB': 1,
              'CC': 2,
              'DD': 1,
              'EE': 2,
              'FF': 3,
              'GG': 4,
              'HH': 5,
              'II': 1,
              'JJ': 2}),
 'BB': defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,
             {'BB': 0,
              'AA': 1,
              'CC': 1,
              'DD': 2,
              'EE': 3,
              'FF': 4,
              'GG': 5,
              'HH': 6,
              'II': 2,
              'JJ': 3}),
 'CC': defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,
             {'CC': 0,
              'AA': 2,
              'BB': 1,
              'DD': 1,
              'EE': 2,
              'FF': 3,
              'GG': 4,
              'HH': 5,
              'II': 3,
              'JJ': 4}),
 'DD': defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,


In [81]:
def tunnel_dfs(graph: dict, current: str, path: defaultdict(int), time_remain: int = 30, time_lim: int = 30) -> list:
    """
    Search through all the possible paths in our volcano network
    
    Params
    ------
    graph: dict
        dict of all Valve nodes which contains release rate and neighbors
        
    source: str
        name of starting Valave node
        
    path: dict
        current path, {valve1: t_remain1, valve2: t_remain2, ...}. If a valve
        is in path, it has been visited in current path
        
    time_remain: int
        time remaining at current node
        
    time_lim: int
        time limit; accounts for 1 min valve opening and 1 min travelling
        between valve nodes
        
    Returns
    -------
    valve_paths: list
        contains dict {valve_id: time_release, ...} which represents each path
        and the order and timing in which the valves were opened
        e.g. if valve 'BB' was reached in minute 1, and opened by minute 2, the
        first entry would be {"BB": 28, ...}
    """
    # print(f'current: {current}\ttime_remain: {time_remain}')
    if not path:
        # print('path is empty; initialize time_remain')
        time_remain = time_lim
        
    # do path recording here
    path[current] = time_remain
    candidates = [node for node in tunnels 
                  if (node not in path.keys()) 
                  and (time_remain >= dists[current][node] + 2)]
    # print(f'neighbors:\n{candidates}')
    if candidates:
        for node in candidates:
            # yield from handles the fact that each tunnel_dfs call has a
            # for node in candidates: yield ...
            # t - dist - 1 accounts for valve opening
            yield from tunnel_dfs(graph, node, path.copy(), time_remain - dists[current][node] - 1, time_lim)
    else:
        # no more viable candidates; end of path reached
        yield path
        

In [106]:
path = defaultdict(int)
paths = tunnel_dfs(graph=tunnels, current='AA', path=path)

In [107]:
totals = {">".join(p.keys()): reduce(
    lambda subtot, valve: subtot + p[valve] * tunnels[valve].rate, 
    p, 0) for p in paths}
print(f"max pressure released: {max(totals.items(), key=lambda p: p[1])}")

max pressure released: ('AA>DD>BB>JJ>HH>EE>CC>FF', 1651)


In [101]:
def get_total_keys(totals):
    yield from totals.keys()

In [104]:
p = get_total_keys(totals)
next(p)

'AA>BB>CC>DD>EE>FF>GG>II>HH'

In [85]:
for path in paths:
    print(path)
    tot = reduce(
    lambda subtot, valve: subtot + path[valve] * tunnels[valve].rate, 
    path, 0)
    print(tot)
    input()

defaultdict(<class 'int'>, {'AA': 30, 'BB': 28, 'CC': 26, 'DD': 24, 'EE': 22, 'GG': 19, 'HH': 17, 'JJ': 9, 'II': 7, 'FF': 2})
1525


 0


defaultdict(<class 'int'>, {'AA': 30, 'BB': 28, 'CC': 26, 'DD': 24, 'EE': 22, 'GG': 19, 'II': 13, 'FF': 8, 'HH': 5})
1072


 0


defaultdict(<class 'int'>, {'AA': 30, 'BB': 28, 'CC': 26, 'DD': 24, 'EE': 22, 'GG': 19, 'II': 13, 'FF': 8, 'JJ': 2})
1004


...

 0


defaultdict(<class 'int'>, {'AA': 30, 'BB': 28, 'CC': 26, 'DD': 24, 'EE': 22, 'GG': 19, 'JJ': 12, 'HH': 4, 'FF': 1})
1302


 0


defaultdict(<class 'int'>, {'AA': 30, 'BB': 28, 'CC': 26, 'DD': 24, 'EE': 22, 'GG': 19, 'JJ': 12, 'II': 10, 'FF': 5, 'HH': 2})
1258


KeyboardInterrupt: Interrupted by user

In [54]:
time_release = 26
time_lim = 30
time_remain = 4
path = {'AA': 29, 'BB': 27}
current = 'BB'
dists[current]

defaultdict(<function __main__.dijkstra_valves.<locals>.<lambda>()>,
            {'BB': 0,
             'AA': 1,
             'CC': 1,
             'DD': 2,
             'EE': 3,
             'FF': 4,
             'GG': 5,
             'HH': 6,
             'II': 2,
             'JJ': 3})

In [55]:
candidates = [node for node in tunnels 
                  if (node not in path.keys()) 
                  and (time_remain >= dists[current][node] + 2)]
candidates

['CC', 'DD', 'II']